In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time


import warnings
warnings.filterwarnings(action='ignore')

sns.set()
#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

pd.options.plotting.backend = "plotly"

In [16]:
# import pandas_profiling

# ----------------- 학습 ----------------------
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# ----------------- 모델 ----------------------
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from xgboost.sklearn import XGBRegressor, XGBClassifier

# ----------------- 평가 ----------------------
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
# from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve

# from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve

# 데이터 로드

In [7]:
train = pd.read_csv('../data/traindf.csv')
test = pd.read_csv('../data/testdf.csv')
train = train.rename(columns={'Unnamed: 0':'날짜'})
test = test.rename(columns={'Unnamed: 0':'날짜'})

# train.info(),test.info()


In [26]:
X_train = train.drop(['날짜','등락률','등락여부'],axis=1)
y_train = train['등락여부']
X_test = test.drop(['날짜','등락률','등락여부'],axis=1)
y_test= test['등락여부']

In [27]:
y_train  =round(y_train)
y_test  =round(y_test)


In [11]:
# pip install optuna

# optuna

In [10]:
import optuna

In [24]:
def my_objective_XGB(trial, data=X_train, target=y_train):

    #train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param = {
        # 'metric': 'rmse',
        'random_state': 1,
        'n_estimators': 100,  #100
        'reg_alpha' : trial.suggest_float('reg_alpha', 1e-2, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10.0),  #reg_alpha=0.0
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01,0.1),
        'max_depth': trial.suggest_categorical('max_depth', [1,30]),
        #'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        #'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        #'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        #'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        #'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = XGBClassifier(**param)

    model.fit(X_train,y_train
              ,eval_set=[(X_test,y_test)]
              ,early_stopping_rounds=10,verbose=False)
    pred = model.predict(X_test)

    score = accuracy_score(y_test, pred)
    return score

In [12]:

study = optuna.create_study(direction='maximize')


# y_train = np.expm1(y_train_log)
# y_test  = np.expm1(y_test_log)


# my_objective_LGBM(trial):
# study.optimize(my_objective_LGBM, n_trials=50)

# def my_objective_LGBM(trial, data=X_train, target=y_train):
study.optimize(lambda trial : my_objective_XGB(trial, y_train,y_test), n_trials=100)

[I 2023-06-21 10:05:03,060] A new study created in memory with name: no-name-0681ae84-f08a-4eff-908f-27c866e56be0
[I 2023-06-21 10:05:03,191] Trial 0 finished with value: 0.6862745098039216 and parameters: {'reg_alpha': 0.36193052811310655, 'reg_lambda': 8.211831927553167, 'learning_rate': 0.013053793741445042, 'max_depth': 1}. Best is trial 0 with value: 0.6862745098039216.
[I 2023-06-21 10:05:03,230] Trial 1 finished with value: 0.7156862745098039 and parameters: {'reg_alpha': 2.966133241869242, 'reg_lambda': 0.3733684635218401, 'learning_rate': 0.0476598239985128, 'max_depth': 30}. Best is trial 1 with value: 0.7156862745098039.
[I 2023-06-21 10:05:03,292] Trial 2 finished with value: 0.6862745098039216 and parameters: {'reg_alpha': 9.769882798783433, 'reg_lambda': 2.4224639070869585, 'learning_rate': 0.011053783131090284, 'max_depth': 30}. Best is trial 1 with value: 0.7156862745098039.
[I 2023-06-21 10:05:03,320] Trial 3 finished with value: 0.7156862745098039 and parameters: {'re

In [ ]:
print('튜닝 횟수 :', len(study.trials))
print("-------------------")
print('Best param:', study.best_trial.params)
print('Best param:', study.best_params)
print("-------------------")
print('Best score:', study.best_trial.value)
print("-------------------")
print('study.trials 전체 정보 :', study.trials)


<pre>
튜닝 횟수 : 50
-------------------
Best param: {'reg_alpha': 2.5594087722267993, 'reg_lambda': 4.62342522497871, 'learning_rate': 0.01867555614418941, 'max_depth': 30}
Best param: {'reg_alpha': 2.5594087722267993, 'reg_lambda': 4.62342522497871, 'learning_rate': 0.01867555614418941, 'max_depth': 30}
-------------------
Best score: 0.7156862745098039

<pre>
튜닝 횟수 : 100
-------------------
Best param: {'reg_alpha': 0.7649224155253016, 'reg_lambda': 2.0595046097238754, 'learning_rate': 0.02071786918763962, 'max_depth': 30}
Best param: {'reg_alpha': 0.7649224155253016, 'reg_lambda': 2.0595046097238754, 'learning_rate': 0.02071786918763962, 'max_depth': 30}
-------------------
Best score: 0.7254901960784313
-------------------

* accuracy_score  0.7157

In [28]:
xgb=XGBClassifier(random_state = 1, reg_alpha= 0.7649224155253016, reg_lambda= 2.0595046097238754, learning_rate= 0.02071786918763962, max_depth= 30)
xgb.fit(X_train,y_train)
# print( xgb.coef_ )

pred = xgb.predict(X_test)
score = accuracy_score(y_test, pred)
print(f"score : {score:.4f}")

score : 0.7157


### f1_score
* sklearn.metrics.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')

In [30]:
score = f1_score(y_test, pred)
print(f"score : {score:.4f}")

score : 0.7339
